In [1]:
%load_ext autoreload
%autoreload 2

import math
import numpy as np

import torch
import torch_geometric

from utils import get_iso_permuted_dataset
from utils import flatten_list

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
print (device)

torch.set_default_dtype(torch.float32)
#torch.set_default_dtype(torch.float64)

cuda


In [3]:
# first, get dataset
datafile = "./2mer-test.pkl"

hhh = "./data/h_s_only_augccpvdz_density.out"
ooo = "./data/o_s_only_augccpvdz_density.out"
ccc = "./data/c_s_only_augccpvdz_density.out"
nnn = "./data/n_s_only_augccpvdz_density.out"
ppp = "./data/p_s_only_augccpvdz_density.out"

dataset = get_iso_permuted_dataset(datafile,h_iso=hhh,c_iso=ccc,n_iso=nnn,o_iso=ooo,p_iso=ppp)

split = 500
b = 1
loader = torch_geometric.data.DataLoader(dataset, batch_size=b, shuffle=False)




FileNotFoundError: [Errno 2] No such file or directory: './2mer-test.pkl'

In [3]:
from e3nn.nn.models.gate_points_2101 import Network
from e3nn import o3

model_kwargs = {
    "irreps_in": "5x 0e", #irreps_in 
    "irreps_hidden": [(mul, l, p) for l, mul in enumerate([200,67,40,29]) for p in [-1, 1]], #irreps_hidden
    #"irreps_hidden": "100x0e + 100x0o",
    "irreps_out": "14x0e + 5x1o + 5x2e + 2x3o + 1x4e", #irreps_out
    "irreps_node_attr": None, #irreps_node_attr
    "irreps_edge_attr": o3.Irreps.spherical_harmonics(3), #irreps_edge_attr
    "layers": 5,
    "max_radius": 3.5,
    "num_neighbors": 12.666666,
    "number_of_basis": 10,
    "radial_layers": 1,
    "radial_neurons": 128,
    "num_nodes": 24,
    "reduce_output": False,
}

model = Network(**model_kwargs)

optim = torch.optim.Adam(model.parameters(), lr=1e-2)
optim.zero_grad()

model.to(device)

Network(
  (layers): ModuleList(
    (0): Compose(
      (first): Convolution(
        (sc): FullyConnectedTensorProduct(5x0e x 1x0e -> 336x0e+67x1o+40x2e+29x3o | 1680 paths | 1680 weights)
        (lin1): FullyConnectedTensorProduct(5x0e x 1x0e -> 5x0e | 25 paths | 25 weights)
        (fc): FullyConnectedNet[10, 128, 20]
        (tp): TensorProduct(5x0e x 1x0e+1x1o+1x2e+1x3o -> 5x0e+5x1o+5x2e+5x3o | 20 paths | 20 weights)
        (lin2): FullyConnectedTensorProduct(5x0e+5x1o+5x2e+5x3o x 1x0e -> 336x0e+67x1o+40x2e+29x3o | 2360 paths | 2360 weights)
      )
      (second): Gate (336x0e+67x1o+40x2e+29x3o -> 200x0e+67x1o+40x2e+29x3o)
    )
    (1): Compose(
      (first): Convolution(
        (sc): FullyConnectedTensorProduct(200x0e+67x1o+40x2e+29x3o x 1x0e -> 472x0e+67x1o+67x1e+40x2o+40x2e+29x3o+29x3e | 101330 paths | 101330 weights)
        (lin1): FullyConnectedTensorProduct(200x0e+67x1o+40x2e+29x3o x 1x0e -> 200x0e+67x1o+40x2e+29x3o | 46930 paths | 46930 weights)
        (fc): FullyCo

In [4]:
checkpoint = torch.load('trainall-400.pkl')

model.load_state_dict(checkpoint)

<All keys matched successfully>

In [5]:
# generate density from CUBE file
from utils import generate_grid, gau2grid_density_kdtree
from ase.io.cube import read_cube

num = 0

data = loader.dataset[num]

# run model on that sample
mask = torch.where(data.y == 0, torch.zeros_like(data.y), torch.ones_like(data.y)).detach()
y_ml = model(data.to(device))*mask.to(device)

#get grid from cube file
# ase converts all distances in cube file to angstroms
f = open("./Dt-1.cube", "r")

cube_dict = read_cube(f)
true_density = cube_dict['data']

origin = cube_dict['origin'] #ang
atoms = cube_dict['atoms']
npts_x = true_density.shape[0]
npts_y = true_density.shape[1]
npts_z = true_density.shape[2]
x_spacing = atoms.cell[0,0]/npts_x #assumes a rectangular grid
y_spacing = atoms.cell[1,1]/npts_y #assumes a rectangular grid
z_spacing = atoms.cell[2,2]/npts_z #assumes a rectangular grid
xlin = np.linspace(origin[0], origin[0]+atoms.cell[0,0]-x_spacing, num=npts_x)
ylin = np.linspace(origin[1], origin[1]+atoms.cell[1,1]-y_spacing, num=npts_y)
zlin = np.linspace(origin[2], origin[2]+atoms.cell[2,2]-z_spacing, num=npts_z)

x,y,z = np.meshgrid(xlin,ylin,zlin,indexing='ij')


In [11]:
# generate density on a grid for a random structure
from utils import generate_grid, gau2grid_density_kdtree

num = 0

data = loader.dataset[num]

# run model on that sample
mask = torch.where(data.y == 0, torch.zeros_like(data.y), torch.ones_like(data.y)).detach()
y_ml = model(data.to(device))*mask.to(device)

x,y,z,vol,x_spacing,y_spacing,z_spacing = generate_grid(data,spacing=0.2,buffer=5.0)

In [6]:
Rs = [(14, 0), (5, 1), (5, 2), (2, 3), (1, 4)]

target_density, ml_density = gau2grid_density_kdtree(x.flatten(),y.flatten(),z.flatten(),data,y_ml,Rs, isoOnlyFlag=0)

angstrom2bohr = 1.8897259886
vol = x_spacing * y_spacing * z_spacing
print("Read in: vol, x, y, z spacing", vol, x_spacing, y_spacing, z_spacing)

num_ele_ml = np.sum(ml_density)*vol*angstrom2bohr**3
num_ele_target = np.sum(target_density)*vol*angstrom2bohr**3
num_ele_true = np.sum(true_density)*vol*angstrom2bohr**3
print("Predicted:", num_ele_ml)
print("Target:", num_ele_target)
print("True:", num_ele_true)
print()

ep_ml = 100 * np.sum(np.abs(ml_density.flatten()-target_density.flatten())) / np.sum(target_density.flatten())
ep_project = 100 * np.sum(np.abs(target_density.flatten()-true_density.flatten())) / np.sum(true_density.flatten())
ep_true = 100 * np.sum(np.abs(ml_density.flatten()-true_density.flatten())) / np.sum(true_density.flatten())
print("ML - target eps: ",ep_ml)
print("Projection eps: ",ep_project)
print("ML - true eps: ",ep_true)



small:  1e-05
Read in: vol, x, y, z spacing 0.001185477689497923 0.10583544211276823 0.10583544211276824 0.10583544211276824
Predicted: 580.9321883056722
Target: 580.9037910464737
True: 581.1240486346064

ML - target eps:  0.6445035997098081
Projection eps:  0.7891244693497593
ML - true eps:  1.0725486813494987


In [7]:
# print out CUSTOM cube file
from ase.io.cube import write_cube
from ase.atoms import Atoms

numbers_out = data.z.flatten().cpu().detach().numpy()
pos_out = data.pos_orig.cpu().detach().numpy()
cell_out = [atoms.cell[0,0], atoms.cell[1,1], atoms.cell[2,2]]
#cell_out = [cell[0,0], cell[1,1], cell[2,2]]

atoms_out = Atoms(numbers=numbers_out, positions=pos_out, cell=cell_out)

target_out = target_density.reshape(x.shape)
ml_out = ml_density.reshape(x.shape)

#f_out = open("4mer.03.target.cube", "w")
#write_cube(f_out, atoms_out, data=target_out, origin=origin)

f_out2 = open("2mertest-ml-0.cube", "w")
write_cube(f_out2, atoms_out, data=ml_out, origin=origin)
